In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from tqdm import tqdm

In [2]:
width = 64
n_cls = 10
mod = resnet50(width_per_group=width)
mod.fc = torch.nn.Linear(2048, n_cls)

In [3]:
transform = transforms.Compose(
    [transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
trainset = torch.load(f'./cifar10_noisyp0.2.pt') # noisy version
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=None)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [4]:
lr = 0.0001
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mod.parameters(), lr=lr)

In [5]:
mod = mod.cuda()
for epoch in range(100):  # loop over the dataset multiple times

    # running_loss = 0.0
    # for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
    with tqdm(trainloader, unit="batch") as tepoch:
      for inputs, labels in tepoch:
        tepoch.set_description(f'Epoch {epoch}')
        
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mod(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # # print statistics
        # running_loss += loss.item()
        tepoch.set_postfix(loss=loss.item())
    # print(f'[{epoch + 1}] loss: {running_loss / len(trainloader):.3f}')

print('Finished Training')
mod = mod.cpu()

Epoch 18:  13%|█▎        | 49/391 [00:01<00:11, 29.16batch/s, loss=1.84]


KeyboardInterrupt: 